In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19284616
paper_name = 'thorsen_tamas_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheets1 = ['0.5 As 24','1 As 24h','1.5 As 24h','0.5As 48h','1 As 48h','1.5 As 48h']
sheets2 = ['Cd 75 24h','Cd 100 24h','Cd 150 24h','Cd 75 48h','Cd 100 48h','Cd 150 48h']

In [9]:
original_data_list1 = []
for s in sheets1:
    original_data = pd.read_excel('raw_data/As(III) list avg.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['orf'] = original_data['Sys name'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data['data'] = original_data['Average M'].astype(float)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    original_data_list1.append(original_data)

Original data dimensions: 4856 x 12
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Australian Result, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 12
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Australian Result, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 12
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Australian Result, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 17923 x 19
             Plate  Row  Column  Average M  Variance  M 1  M 2  M 3  M 4  \
index_input                                                                
4856           NaN  NaN     NaN        NaN       NaN  NaN  NaN  NaN  NaN   
4857           NaN  NaN     NaN        NaN       NaN  NaN  NaN  NaN  NaN   
4858           NaN  NaN     NaN        NaN       NaN  NaN  NaN  NaN  NaN   
4859           NaN 

In [10]:
original_data_list2 = []
for s in sheets2:
    original_data = pd.read_excel('raw_data/Cd list avg.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['orf'] = original_data['Sys name'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data['data'] = original_data['Average M'].astype(float)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    original_data_list2.append(original_data)

Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index: []
(4761, 1)
Original data dimensions: 4856 x 11
Empty DataFrame
Columns: [Plate, Row, Column, Average M, Variance, M 1, M 2, M 3, M 4, Sys name, Gene name, orf]
Index:

In [11]:
original_data1 = pd.concat(original_data_list1, axis=1)
original_data2 = pd.concat(original_data_list2, axis=1)

In [14]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [16]:
original_data.shape

(4761, 12)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [1320, 5362, 5363, 5364, 5365, 5366, 1319, 5367, 5368, 5369, 5370, 5371]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,1320,5362,5363,5364,5365,5366,1319,5367,5368,5369,5370,5371
data_type,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,
YAL002W,-0.187,0.216,1.781,0.329,0.470,0.951,-0.553,1.929,2.532,2.999,1.568,3.405
YAL004W,0.113,-0.123,-0.343,-0.027,-0.218,-0.145,0.157,-0.034,-0.075,0.040,-0.021,-0.237
YAL005C,-0.178,0.019,-0.109,0.161,0.064,0.133,-1.631,-0.024,-0.229,-0.152,0.053,-0.142
YAL007C,-0.061,-0.127,-0.229,-0.275,-0.246,-0.256,0.190,-0.007,0.038,0.147,-0.117,-0.057
YAL008W,0.185,0.113,0.032,-0.048,0.183,0.015,0.069,-0.035,0.049,-0.056,-0.042,0.006


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1320,5362,5363,5364,5365,5366,1319,5367,5368,5369,5370,5371
,data_type,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,
2,YAL002W,-0.187,0.216,1.781,0.329,0.470,0.951,-0.553,1.929,2.532,2.999,1.568,3.405
1863,YAL004W,0.113,-0.123,-0.343,-0.027,-0.218,-0.145,0.157,-0.034,-0.075,0.040,-0.021,-0.237
4,YAL005C,-0.178,0.019,-0.109,0.161,0.064,0.133,-1.631,-0.024,-0.229,-0.152,0.053,-0.142
5,YAL007C,-0.061,-0.127,-0.229,-0.275,-0.246,-0.256,0.190,-0.007,0.038,0.147,-0.117,-0.057
6,YAL008W,0.185,0.113,0.032,-0.048,0.183,0.015,0.069,-0.035,0.049,-0.056,-0.042,0.006


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,1320,5362,5363,5364,5365,5366,1319,5367,5368,5369,...,5363,5364,5365,5366,1319,5367,5368,5369,5370,5371
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.187,0.216,1.781,0.329,0.470,0.951,-0.553,1.929,2.532,2.999,...,1.796995,0.936511,0.899509,1.179075,-0.678684,2.548059,3.210308,5.497798,2.824220,5.514205
1863,YAL004W,0.113,-0.123,-0.343,-0.027,-0.218,-0.145,0.157,-0.034,-0.075,0.040,...,-0.299250,0.055847,-0.539198,-0.150431,0.268928,0.024118,0.016483,0.051205,-0.030542,-0.389167
4,YAL005C,-0.178,0.019,-0.109,0.161,0.064,0.133,-1.631,-0.024,-0.229,-0.152,...,-0.068308,0.520917,0.050504,0.186798,-2.117455,0.036975,-0.172182,-0.302207,0.102405,-0.235180
5,YAL007C,-0.061,-0.127,-0.229,-0.275,-0.246,-0.256,0.190,-0.007,0.038,0.147,...,-0.186740,-0.557649,-0.597751,-0.285080,0.312972,0.058833,0.154919,0.248158,-0.203013,-0.097402
6,YAL008W,0.185,0.113,0.032,-0.048,0.183,0.015,0.069,-0.035,0.049,-0.056,...,0.070850,0.003898,0.299351,0.043658,0.151478,0.022832,0.168395,-0.125501,-0.068270,0.004715


# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 19284616...


  0%|          | 0/12 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 12/12 [01:42<00:00,  8.52s/it]

Updating the data_modified_on field...
